In [1]:

import pandas as pd
import numpy as np
# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader, TensorDataset

from helper.model_plots import *

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA



In [2]:

daily = pd.read_parquet('data/daily.parquet',engine='fastparquet')
labeled_users = pd.read_parquet('data/labeled_users.parquet',engine='fastparquet')


In [3]:
target_cluster_users = labeled_users[labeled_users['cluster_label'] == 0]['user_id'].to_list()


In [4]:

target_cluster_users[:5]


[44, 46, 5977, 7612, 9388]

In [5]:
target_cluster_users_days = daily[daily['user_id'].isin(target_cluster_users)]


In [6]:
target_cluster_users_days.head()


,user_id,day_of_year,odds_log_mean,odds_log_std,bet_prop_log_mean,bet_prop_log_std,n_sessions,n_active_hours,bet_top1hour%,bet_top2hours%,bet_top4hours%,bet_mean_abs_dev_uniform%,odd_top1hour%,odd_top2hours%,odd_top4hours%,odd_mean_abs_dev_uniform%,slope_bet_day,slope_odd_day,n_bet_proportion
0,44,253,0.045360,0.027368,0.000000,0.000000,1.0,2,0.500000,1.000000,1.000000,0.076389,0.505995,1.000000,1.000000,0.076389,0.000000,0.001335,1.000000
1,44,254,0.537515,0.259239,1.242422,0.736760,1.0,2,0.825483,1.000000,1.000000,0.076389,0.515239,1.000000,1.000000,0.076389,-0.029744,-0.013300,1.666667
2,44,255,0.029559,0.000000,-0.069071,0.000000,1.0,1,1.000000,1.000000,1.000000,0.079861,1.000000,1.000000,1.000000,0.079861,-0.000210,0.000090,0.312500
3,44,257,0.036531,0.027343,0.324303,0.698184,3.0,3,0.559135,0.782729,0.782729,0.072917,0.338710,0.670968,0.670968,0.072917,0.001503,0.000293,1.342282
4,44,259,0.095418,0.104053,-0.068377,0.002970,1.0,1,1.000000,1.000000,1.000000,0.079861,1.000000,1.000000,1.000000,0.079861,-0.009365,0.013068,4.866970


In [7]:
target_cluster_users_days.columns


Index(['user_id', 'day_of_year', 'odds_log_mean', 'odds_log_std',
       'bet_prop_log_mean', 'bet_prop_log_std', 'n_sessions', 'n_active_hours',
       'bet_top1hour%', 'bet_top2hours%', 'bet_top4hours%',
       'bet_mean_abs_dev_uniform%', 'odd_top1hour%', 'odd_top2hours%',
       'odd_top4hours%', 'odd_mean_abs_dev_uniform%', 'slope_bet_day',
       'slope_odd_day', 'n_bet_proportion'],
      dtype='object')

In [8]:
target_days = target_cluster_users_days.drop(columns=['user_id','day_of_year','odd_mean_abs_dev_uniform%', 'odd_top1hour%', 'odd_top2hours%', 'odd_top4hours%','bet_mean_abs_dev_uniform%'])


In [9]:
target_days.head()


,odds_log_mean,odds_log_std,bet_prop_log_mean,bet_prop_log_std,n_sessions,n_active_hours,bet_top1hour%,bet_top2hours%,bet_top4hours%,slope_bet_day,slope_odd_day,n_bet_proportion
0,0.045360,0.027368,0.000000,0.000000,1.0,2,0.500000,1.000000,1.000000,0.000000,0.001335,1.000000
1,0.537515,0.259239,1.242422,0.736760,1.0,2,0.825483,1.000000,1.000000,-0.029744,-0.013300,1.666667
2,0.029559,0.000000,-0.069071,0.000000,1.0,1,1.000000,1.000000,1.000000,-0.000210,0.000090,0.312500
3,0.036531,0.027343,0.324303,0.698184,3.0,3,0.559135,0.782729,0.782729,0.001503,0.000293,1.342282
4,0.095418,0.104053,-0.068377,0.002970,1.0,1,1.000000,1.000000,1.000000,-0.009365,0.013068,4.866970


In [11]:
scaler = StandardScaler()
scaled_target = scaler.fit_transform(target_days)
scaled_target[0]


array([-1.36516166e+00, -9.65511597e-01,  3.82051091e-01, -7.29895406e-01,
       -4.23849778e-01,  3.40135127e-01, -2.58318793e+00,  2.02560233e-01,
        2.02560233e-01,  1.23019181e-03,  3.12928023e-02, -1.23900897e-01])

In [ ]:

original_target = scaler.inverse_transform(scaled_target)
original_target[0]


array([4.53595407e-02, 2.73677334e-02, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 2.00000000e+00, 5.00000000e-01, 1.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 1.33508292e-03, 1.00000000e+00])